In [ ]:

!pip install transformers[torch]
!pip install accelerate -U

In [ ]:
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler,TensorDataset,Dataset
from transformers import BertTokenizer, BertForSequenceClassification,Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import re,string
from nltk.stem import WordNetLemmatizer

In [ ]:
df_train=pd.read_csv('/content/train.csv',encoding='latin-1')

In [ ]:
df_train=df_train[['text','sentiment']]

In [ ]:
df_train['text'] = df_train['text'].astype(str)

In [ ]:
df_train['sentiment']=df_train['sentiment'].replace({'positive':0,'negative':1,'neutral':2})

In [ ]:
df_train['sentiment'].value_counts()

sentiment
2    11118
0     8582
1     7781
Name: count, dtype: int64

In [ ]:
def clean_text(text):
    text = str(text).lower() ## Make text lowercase
    text = re.sub('\[.*?\]', '', text) ## remove any sequence of characters in square brackets
    text = re.sub('https?://\S+|www\.\S+', '', text) ## remove links
    text = re.sub('<.*?>+', '', text) ## remove HTML tags
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) ## remove punctuation
    text = re.sub('\n', '', text) ## remove newline characters
    text = re.sub('\w*\d\w*', '', text) ## remove words containing numbers
    text = re.sub(r'[^a-z/A-Z/0-9/ ]', '', text)
    return text

In [ ]:
df_train['text']=df_train['text'].apply(clean_text)

In [ ]:
text=df_train.text.values
sentiment=df_train.sentiment.values

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize(df_train['text'][0]))

[8909, 2031, 5838, 2065, 1045, 2020, 2183]

In [ ]:
max_len=0
for i in range(len(df_train)):
  if len(df_train['text'][i].split())>max_len:
         max_len = len(df_train['text'][i].split())

In [ ]:
max_len

32

In [ ]:
def preprocessing(input_text,tokenizer):
  return tokenizer.encode_plus(input_text,
                               add_special_tokens=True,
                               max_length=32,pad_to_max_length=True,return_attention_mask=True)

In [ ]:
input_ids=[]
attention_masks=[]
for tweet in text:
  encoded=preprocessing(tweet,tokenizer)
  input_ids.append(encoded['input_ids'])
  attention_masks.append(encoded['attention_mask'])


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
input_ids = torch.tensor(input_ids)
attention_masks = torch.tensor(attention_masks)
labels = torch.tensor(sentiment)

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# new_sentence='Before I get too distracted, I`d like to thank my new followers for taking the trouble to follow me! And to my others: feelin the love'
# # We need Token IDs and Attention Mask for inference on the new sentence
# test_ids = []
# test_attention_mask = []

# # Apply the tokenizer
# encoding = preprocessing(new_sentence, tokenizer)

# # Extract IDs and Attention Mask
# test_ids.append(encoding['input_ids'])
# test_attention_mask.append(encoding['attention_mask'])
# test_ids = torch.tensor(test_ids)
# test_attention_mask = torch.tensor(test_attention_mask)

# # Forward pass, calculate logit predictions
# with torch.no_grad():
#   output = model(test_ids, token_type_ids = None, attention_mask = test_attention_mask)
#   if np.argmax(output.logits.cpu().numpy())==0:
#     print('positive')
#   elif np.argmax(output.logits.cpu().numpy())==1:
#     print('negative')
#   else:
#     print('neutral')


Checking the Bert Model without fine-tuning

In [ ]:
# new_sentence='Before I get too distracted, I`d like to thank my new followers for taking the trouble to follow me! And to my others: feelin the love'
# # We need Token IDs and Attention Mask for inference on the new sentence
# test_ids = []
# test_attention_mask = []

# # Apply the tokenizer
# encoding = preprocessing(new_sentence, tokenizer)

# # Extract IDs and Attention Mask
# test_ids.append(encoding['input_ids'])
# test_attention_mask.append(encoding['attention_mask'])
# test_ids = torch.tensor(test_ids)
# test_attention_mask = torch.tensor(test_attention_mask)

for i in range(50):
  # Forward pass, calculate logit predictions
  with torch.no_grad():
    output = model(input_ids[i].unsqueeze(0), token_type_ids = None, attention_mask = attention_masks[i].unsqueeze(0))
    if np.argmax(output.logits.cpu().numpy())==0:
      print('positive',tokenizer.convert_ids_to_tokens(input_ids[i]))
    elif np.argmax(output.logits.cpu().numpy())==1:
      print('negative',tokenizer.convert_ids_to_tokens(input_ids[i]))
    else:
      print('neutral',tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[i])))


neutral [CLS] id have responded if i were going [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
neutral [CLS] sooo sad i will miss you here in san diego [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
negative ['[CLS]', 'my', 'boss', 'is', 'bullying', 'me', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
negative ['[CLS]', 'what', 'interview', 'leave', 'me', 'alone', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
neutral [CLS] sons of why couldnt they put

Fine-Tuning

In [ ]:
val_ratio =0.2
batch_size=32

train_ids, val_ids = train_test_split(np.arange(len(input_ids)), random_state=42, test_size=val_ratio)
train_set = TensorDataset(input_ids[train_ids],attention_masks[train_ids],labels[train_ids])
val_set= TensorDataset(input_ids[val_ids],attention_masks[val_ids],labels[val_ids])
train_dataloader = DataLoader(train_set, sampler=RandomSampler(train_set), batch_size=batch_size)
val_dataloader = DataLoader(val_set, sampler=SequentialSampler(val_set), batch_size=batch_size)

In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)


In [ ]:


class MyDataset(Dataset):
    def __init__(self, tensor_dataset):
        self.tensor_dataset = tensor_dataset

    def __getitem__(self, idx):
        input_ids, attention_mask, labels = self.tensor_dataset[idx]
        return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

    def __len__(self):
        return len(self.tensor_dataset)

# Usage:
# Assuming `train_tensor_dataset` and `eval_tensor_dataset` are your existing TensorDatasets
train_set = MyDataset(train_set)
val_set = MyDataset(val_set)


In [ ]:
trainer = Trainer(model=model,
                  args = training_args,
                  train_dataset = train_set,
                  eval_dataset= val_set)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.786400


In [ ]:
trainer.model.save_pretrained(new_model)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# !huggingface-cli login

# model.push_to_hub("Naren579/BERT-sentiment-classification", check_pr=True)

# tokenizer.push_to_hub("Naren579/BERT-sentiment-classification",check_pr=True)